In [ ]:
import opensmile
import librosa
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
import joblib

c:\Users\jbkee\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ravdess_folder = Path(r"C:/Users/jbkee/Desktop/Jupyter-Projects/RAVDESS/audio_speech_actors_01-24")
ravdess_files = ravdess_folder.glob("**/*.wav")
savee_folder = Path(r"C:/Users/jbkee/Desktop/Jupyter-Projects/SAVEE/AudioData")
savee_files = savee_folder.glob("**/*.wav")
emodb_folder = Path(r"C:/Users/jbkee/Desktop/Jupyter-Projects/EMODB")
emodb_files = emodb_folder.glob("*.wav")

In [3]:
def extract_emotion_from_ravdess_filename(filename):
    emotion_map = {
        "01": "neutral",
        "02": "calm",
        "03": "happy",
        "04": "sad",
        "05": "angry",
        "06": "fearful",
        "07": "disgust",
        "08": "surprised",
    }
    
    emotion = filename.split("-")[2]
    
    return emotion_map.get(emotion, "unknown")

def extract_emotion_from_savee_filename(filename):
    emotion_map = {
        "a": "angry",
        "d": "disgust",
        "f": "fearful",
        "h": "happy",
        "n": "neutral",
        "sa": "sad",
        "su": "surprised",
    }
    
    emotion = filename[0].lower()
    
    if filename[:2].lower() in emotion_map:
        return emotion_map[filename[:2].lower()]
    return emotion_map.get(emotion, "unknown")

def extract_emotion_from_emodb_filename(filename):
    emotion_map = {
        "w": "angry",
        "e": "disgust",
        "a": "fearful",
        "f": "happy",
        "n": "neutral",
        "t": "sad",
    }
    
    emotion = filename[5].lower()

    return emotion_map.get(emotion, "unknown")

clusters = {
    "angry": "negative",
    "disgust": "negative",
    "fearful": "negative",
    "happy": "positive",
    "neutral": "neutral",
    "sad": "negative",
    "surprised": "positive",
    "calm": "neutral",
}

In [ ]:
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.emobase,
    feature_level=opensmile.FeatureLevel.Functionals,
)

def process_file(audio_path):
    y, sr = librosa.load(audio_path, sr=16000, mono=True)
    
    # # Normalize
    y = y / np.max(np.abs(y)) if np.max(np.abs(y)) > 0 else y

    #Extract features using opensmile
    return smile.process_signal(y, sr)

In [5]:
rows = []

print("Processing SAVEE Files")
for file in savee_files:
    emotion = extract_emotion_from_savee_filename(file.name)
    emotion = clusters.get(emotion, "unknown")
    if emotion != "unknown":
        df_feat = process_file(file)
        df_feat["emotion"] = emotion
        df_feat["dataset"] = "SAVEE"
        df_feat["actor"] = file.parent.name
        rows.append(df_feat)
        print(file.name + " processed. Emotion: " + emotion)
    else:
        print("Unknown emotion for: " + file.name)

print("Processing EMODB Files")
for file in emodb_files:
    emotion = extract_emotion_from_emodb_filename(file.name)
    emotion = clusters.get(emotion, "unknown")
    if emotion != "unknown":
        df_feat = process_file(file)
        df_feat["emotion"] = emotion
        df_feat["dataset"] = "EMODB"
        df_feat["actor"] = str(file.name[:2])
        rows.append(df_feat)
        print(file.name + " processed. Emotion: " + emotion)
    else:
        print("Unknown emotion for: " + file.name)

print("Processing RAVDESS Files")
for file in ravdess_files:
    emotion = extract_emotion_from_ravdess_filename(file.name)
    emotion = clusters.get(emotion, "unknown")
    if emotion != "unknown":
        df_feat = process_file(file)
        df_feat["emotion"] = emotion
        df_feat["dataset"] = "RAVDESS"
        df_feat["actor"] = file.parent.name
        rows.append(df_feat)
        print(file.name + " processed. Emotion: " + emotion)
    else:
        print("Unknown emotion for: " + file.name)

features = pd.concat(rows, axis=0).reset_index(drop=True)

df = pd.DataFrame(features)

Processing SAVEE Files
a01.wav processed. Emotion: negative
a02.wav processed. Emotion: negative
a03.wav processed. Emotion: negative
a04.wav processed. Emotion: negative
a05.wav processed. Emotion: negative
a06.wav processed. Emotion: negative
a07.wav processed. Emotion: negative
a08.wav processed. Emotion: negative
a09.wav processed. Emotion: negative
a10.wav processed. Emotion: negative
a11.wav processed. Emotion: negative
a12.wav processed. Emotion: negative
a13.wav processed. Emotion: negative
a14.wav processed. Emotion: negative
a15.wav processed. Emotion: negative
d01.wav processed. Emotion: negative
d02.wav processed. Emotion: negative
d03.wav processed. Emotion: negative
d04.wav processed. Emotion: negative
d05.wav processed. Emotion: negative
d06.wav processed. Emotion: negative
d07.wav processed. Emotion: negative
d08.wav processed. Emotion: negative
d09.wav processed. Emotion: negative
d10.wav processed. Emotion: negative
d11.wav processed. Emotion: negative
d12.wav process

In [8]:
temp = df.copy()
temp_x = temp.drop(columns=["emotion", "dataset", "actor"])
temp_y = temp["emotion"]

scaler = StandardScaler().fit_transform(temp_x)
joblib.dump(scaler, "emobase_edsa.pkl")

final = pd.DataFrame(scaler, columns=temp_x.columns)
final = final.join(temp_y)

final.to_csv("emobase_edsa.csv", index=False)
print("All features saved to emobase_edsa.csv")

All features saved to emobase_edsa.csv


In [7]:
print(features.shape)

(2374, 991)
